In [7]:
import utils
import librosa
import os
import IPython.display as ipd

AUDIO_DIR = os.environ.get('AUDIO_DIR')

filename = utils.get_audio_path(AUDIO_DIR, 99134)
print('File: {}'.format(filename))

x, sr = librosa.load(filename, sr=None, mono=True)
print('Duration: {:.2f}s, {} samples'.format(x.shape[-1] / sr, x.size))

start, end = 7, 17
ipd.Audio(data=x[start*sr:end*sr], rate=sr)

File: D:\Code\Desktop\IIT\Courses\Spring 2020\Deep Learning\FInal Project\github_project\music_analysis_fp\fma_small\099\099134.mp3


NoBackendError: 

In [1]:
import time
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import IPython.display as ipd
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
import tensorflow.keras
from tensorflow.keras import models
# from tensorflow.keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D, Flatten, Reshape
from tensorflow.keras import layers
from tensorflow.keras import optimizers

from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multiclass import OneVsRestClassifier

import utils
import librosa


import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank

def plot_signals(signals):
    fig, axes = plt.subplots(nrows=2, ncols=4, sharex=False,
                             sharey=True, figsize=(20,5))
    fig.suptitle('Time Series', size=16)
    i = 0
    for x in range(2):
        for y in range(4):
            axes[x,y].set_title(list(signals.keys())[i])
            axes[x,y].plot(list(signals.values())[i])
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i += 1

def plot_fft(fft):
    fig, axes = plt.subplots(nrows=2, ncols=4, sharex=False,
                             sharey=True, figsize=(20,4))
    fig.suptitle('Fourier Transforms', size=16)
    i = 0
    for x in range(2):
        for y in range(4):
            data = list(fft.values())[i]
            Y, freq = data[0], data[1]
            axes[x,y].set_title(list(fft.keys())[i])
            axes[x,y].plot(freq, Y)
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i += 1

def plot_fbank(fbank):
    fig, axes = plt.subplots(nrows=2, ncols=4, sharex=False,
                             sharey=True, figsize=(20,4))
    fig.suptitle('Filter Bank Coefficients', size=16)
    i = 0
    for x in range(2):
        for y in range(4):
            axes[x,y].set_title(list(fbank.keys())[i])
            axes[x,y].imshow(list(fbank.values())[i],
                    cmap='hot', interpolation='nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i += 1

def plot_mfccs(mfccs):
    fig, axes = plt.subplots(nrows=2, ncols=4, sharex=False,
                             sharey=True, figsize=(20,4))
    fig.suptitle('Mel Frequency Cepstrum Coefficients', size=16)
    i = 0
    for x in range(2):
        for y in range(4):
            axes[x,y].set_title(list(mfccs.keys())[i])
            axes[x,y].imshow(list(mfccs.values())[i],
                    cmap='hot', interpolation='nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i += 1


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
AUDIO_DIR = os.environ.get('AUDIO_DIR')

tracks = utils.load('fma_metadata/tracks.csv')
features = utils.load('fma_metadata/features.csv')
echonest = utils.load('fma_metadata/echonest.csv')

np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

tracks.shape, features.shape, echonest.shape

((106574, 52), (106574, 518), (13129, 249))

In [ ]:
subset = tracks.index[tracks['set', 'subset'] == 'small']

assert subset.isin(tracks.index).all()
assert subset.isin(features.index).all()

features_all = features.join(echonest, how='inner').sort_index(axis=1)
print('Not enough Echonest features: {}'.format(features_all.shape))

tracks = tracks.loc[subset]
features_all = features.loc[subset]

tracks.shape, features_all.shape

In [9]:
train = tracks[tracks['set', 'split'] == 'training']
val = tracks[tracks['set', 'split'] == 'validation']
test = tracks[tracks['set', 'split'] == 'test']

print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))

# genres = list(LabelEncoder().fit(tracks['track', 'genre_top']).classes_)
genres = list(tracks['track', 'genre_top'].unique())
print('Top genres ({}): {}'.format(len(genres), genres))
# print(train.loc[99135])
# print(train.loc[99134])

84353 training examples, 10958 validation examples, 11263 testing examples
Top genres (17): ['Hip-Hop', 'Pop', nan, 'Rock', 'Experimental', 'Folk', 'Jazz', 'Electronic', 'Spoken', 'International', 'Soul-RnB', 'Blues', 'Country', 'Classical', 'Old-Time / Historic', 'Instrumental', 'Easy Listening']


In [ ]:
filename = utils.get_audio_path(AUDIO_DIR, 2)
print('File: {}'.format(filename))

x, sr = librosa.load(filename, sr=None, mono=True)
print('Duration: {:.2f}s, {} samples'.format(x.shape[-1] / sr, x.size))

start, end = 7, 17
ipd.Audio(data=x[start*sr:end*sr], rate=sr)

In [ ]:
all_tracks = tracks.copy()
all_tracks.columns = ['_'.join(col).strip() for col in all_tracks.columns.values]
print(all_tracks.columns)

In [ ]:
print(all_tracks.shape)

In [ ]:
classes = list(np.unique(all_tracks['track_genre_top']))
print(classes)
class_dist = all_tracks.groupby(['track_genre_top'])['track_duration'].mean().dropna()
print(class_dist)

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Class distribution', y=1.08)
ax.pie(class_dist,labels=class_dist.index,autopct="%1.1f%%",shadow=False,startangle=90)
ax.axis('equal')
plt.show()
all_tracks.reset_index(inplace=True)

In [ ]:
def calc_fft(y, rate):
    n = len(y)
    freq = np.fft.rfftfreq(n, d = 1/rate)
    Y = abs(np.fft.rfft(y)/n)
    return (Y,freq)
    

In [ ]:
def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10),min_periods=1,center=True).mean()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask
# Directory where mp3 are stored.
AUDIO_DIR = os.environ.get('AUDIO_DIR')
print(AUDIO_DIR)

signals = {}
fft = {}
fbank = {}
mfccs = {}

for c in classes:
    mp3_file = all_tracks[all_tracks.track_genre_top == c].iloc[0,0]
    signal, rate = utils.FfmpegLoader().load(utils.get_audio_path(AUDIO_DIR, mp3_file))
#     mask = envelope(signal, rate,0.5)
#     signal = signal[mask]
    print(f'signal -> {signal.shape}')
    print(f'rate -> {rate}')
    print(f'signal rate -> {signal[:rate].shape}')
    signals[c] = signal
    fft[c] = calc_fft(signal,rate)
    
    bank = logfbank(signal[:rate],rate,nfilt=26,nfft=1103).T
    fbank[c] = bank
    mel = mfcc(signal[:rate],rate,numcep=13, nfilt=26, nfft=1103).T
    mfccs[c] = mel

print(rate)

plot_signals(signals)
plt.show()

plot_fft(fft)
plt.show()

plot_fbank(fbank)
plt.show()

plot_mfccs(mfccs)
plt.show()

In [56]:
# CLEANING CORRUPTED FILES
import utils
import librosa
import os
import IPython.display as ipd


subset_small = tracks.index[tracks['set', 'subset'] == 'small']
subset_small = tracks.loc[subset_small]

subset_small.columns = ['_'.join(col).strip() for col in subset_small.columns.values]
print(np.asarray(subset_small.index))
classes = list(np.unique(subset_small['track_genre_top']))
print(classes)
class_dist = subset_small.groupby(['track_genre_top'])['track_duration'].mean().dropna()
print(class_dist)
tids = np.asarray(subset_small.index)

AUDIO_DIR = os.environ.get('AUDIO_DIR')
tids = [99134]
tids = np.asarray(tids)
print(f'tids shape before -> {tids.shape}')
error_tids = []
for tid in tids:
    filename = utils.get_audio_path(AUDIO_DIR, tid)
    try:
        x, sr = librosa.load(filename, sr=None, mono=True)
        print(f'Loaded file {tid}\n')
    except Exception as e:
        print(f'Error loading -> {tid}\n')
        error_tids.append(tid)
        tids = np.delete(tids, np.argwhere(tids==tid))
        continue
print(error_tids)
subset_small.drop(error_tids,inplace = True)
print(f'Deleted in metadata {subset_small.loc[99134]}')
print(f'tids shape after -> {tids.shape}')

    

[     2      5     10 ... 154413 154414 155066]
['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']
track_genre_top
Electronic       251.253
Experimental     238.739
Folk             232.537
Hip-Hop          201.564
Instrumental     205.460
International    252.781
Pop              221.207
Rock             227.191
Name: track_duration, dtype: float64
tids shape before -> (1,)
Error loading -> 99134

[99134]


c:\users\bisar\appdata\local\programs\python\python37\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


KeyError: 99134